In [1]:
%matplotlib inline
import re
from pathlib import Path

import numpy as np

from spectrometer import FID1D, plot, process

Did not find any configuration file. Using defaults.


In [ ]:
# Load raw *.fid data
fids: list[FID1D] = []
for file in Path("../data/some_folder/").glob("*.fid"):
    fids.append(FID1D.from_file(file))

# Extract pulse lengths from metadata
pattern = r"delay_tau=(.*)us,repetition"
delay_tau_us = [float(re.search(pattern, str(fid.pulse))) for fid in fids]

In [ ]:
# Process
signal_strengths = []
for _i, fid in enumerate(fids):
    x, y = fid.simple_fft(phase_shift_kwargs=False)
    y2 = np.abs(y) ** 2
    integral = np.trapz(x=x, y=y2)
    signal_strengths.append(integral)

In [ ]:
# Sort data by delay_tau
delay_tau_us, signal_strengths = zip(*sorted(zip(delay_tau_us, signal_strengths)))

# Plot raw data
cm = 1 / 2.54
fig, axes = plot.subplots(
    figsize=(16.0 * cm, 8.0 * cm),  # Wide Layout: ~16cm, Margin Layout: 10.7cm
)
axes.plot(
    delay_tau_us,
    signal_strengths,
    linestyle="",
    marker="o",
    markersize=0.8,
)
axes.set_ylabel("Signal strength [a.u.]")
axes.set_xlabel("Delay τ [μs]")

plot.adjust_for_thesis(axes)

fig.savefig("t2_decay.pdf")

In [ ]:
# Try to plot simple least squares fit
fit = process.fit_exp_decay(delay_tau_us, signal_strengths)
delay_tau_us_fine = np.linspace(
    delay_tau_us[0], delay_tau_us[-1], len(delay_tau_us) * 10
)
axes.plot(
    delay_tau_us_fine,
    fit["function"](delay_tau_us_fine),
    linestyle="-",
    linewidth=2,
)

# Save plot
fig.savefig("t2_decay_fit.pdf")
fig